In [ ]:
import pandas as pd
import glob
import os
import gc
from sklearn.utils import shuffle


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def merge(input1, input2, output):
    df = pd.read_csv(input1)
    df1 = pd.read_csv(input2)
    
    df = pd.concat([df, df1])
    
    df = df.sample(frac=1).reset_index(drop=True)
    
    df.to_csv(output, index=False)

def merge_from_parquet(folder_path, output):
    search_pattern = os.path.join(folder_path, '*.parquet')
    parquet_files = glob.glob(search_pattern)

    if not parquet_files:
        print(f"Lỗi: Không tìm thấy file .parquet nào trong '{folder_path}'.")
        return
    
    header = True
    for file in parquet_files:
        tmp_df = pd.read_parquet(file)
        tmp_df.to_csv(output, mode='a', index=False, header=header)
        header=False
        gc.collect()
    # print(f"Tìm thấy {len(parquet_files)} file parquet. Bắt đầu đọc và gộp...")
    # all_dfs = [pd.read_parquet(f) for f in parquet_files]
    # full_df = pd.concat(all_dfs, ignore_index=True)
    # print(f"Đã gộp thành công. Tổng số mẫu: {len(full_df)}")
    gc.collect()
    

def splitIL(input, output):
    search_pattern = os.path.join(input, '*.parquet')
    parquet_files = glob.glob(search_pattern)

    if not parquet_files:
        print(f"Lỗi: Không tìm thấy file .parquet nào trong '{input}'.")
        return
    
    all_dfs=[pd.read_parquet(f) for f in parquet_files]
    df = pd.concat(all_dfs, ignore_index=True)

    classes = sorted(df['label'].unique())
    # print(classes[:2])
    
    base_classes = classes[:1]
    new_classes = classes[1:]

    # config
    B_total = 100000
    N_new = 20000

    replay_per_class = lambda c: int(B_total / c)

    sample0= df[df['label'].isin([0,1])].groupby("label").apply(lambda x: x.sample(100000)).reset_index(drop=True)
    sample0 = shuffle(sample0)
    sample0.to_csv(output[0], index=False)
    
    del(sample0)
    gc.collect()
    
    for step, new_cls in enumerate(new_classes, start=1):
        old_cls = base_classes + new_classes[:step]
        # print("OLD CLASS: ", old_cls)
        E = replay_per_class(len(old_cls))
        
        old_samples = df[df["label"].isin(old_cls)].groupby("label").apply(lambda x: x.sample(E)).reset_index(drop=True)
        new_samples = df[df["label"] == new_cls].sample(N_new)
        
        incremental_df = shuffle(pd.concat([old_samples, new_samples]))
        incremental_df.to_csv(output[step], index=False)
        
        gc.collect()
    
input  = "C:/Users/hoang/Documents/Dataset_KLTN/scaled_output_parquet"

input1 = "C:/Users/hoang/Documents/Dataset_KLTN/training_dataset.csv"
input2 = "C:/Users/hoang/Documents/Dataset_KLTN/test_dataset.csv"
output = "C:/Users/hoang/Documents/Dataset_KLTN/merge.csv"
outputIL = ["session0.csv", "session1.csv", "session2.csv", "session3.csv"]

outputIL = ["C:/Users/hoang/Documents/Dataset_KLTN/"+ path for path in outputIL]

print(outputIL)
splitIL(input, outputIL)

gc.collect()

# df = pd.read_csv(output)
# print(df.head())


['C:/Users/hoang/Documents/Dataset_KLTN/session0.csv', 'C:/Users/hoang/Documents/Dataset_KLTN/session1.csv', 'C:/Users/hoang/Documents/Dataset_KLTN/session2.csv', 'C:/Users/hoang/Documents/Dataset_KLTN/session3.csv']


C:\Users\hoang\AppData\Local\Temp\ipykernel_6988\4058277735.py:66: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample0= df[df['label'].isin([0,1])].groupby("label").apply(lambda x: x.sample(100000)).reset_index(drop=True)
C:\Users\hoang\AppData\Local\Temp\ipykernel_6988\4058277735.py:78: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  old_samples = df[df["label"].isin(old_cls)].groupby("label").apply(lambda x: x.samp

IndexError: list index out of range

In [1]:
import os, sys
from pathlib import Path

import pandas as pd 
import dask.dataframe as dd

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

import hashlib  
import ipaddress
import json
import glob
import gc

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

dir_in = "C:/Users/hoang/Documents/Dataset_KLTN/ciciot2023_extracted/merge-processed/merge1.3/part.0.parquet"
dir_out = "C:/Users/hoang/Documents/Dataset_KLTN/ciciot2023_extracted/merge-processed/merge1.3/part.1.parquet"

df = pd.read_parquet(dir_in)

dtypes = {}    
with open('features.json') as json_file:
    data = json.load(json_file)
    for key, type in data.items():
        if type == "int8":
            dtypes[key]= np.int8
        elif type == "float32":
            dtypes[key] = np.float32
    
    json_file.close()

print(dtypes)

def astype(df):
    for key, type in data.items():
        if type == "int8":
            df[key] = df[key].astype(np.int8)
        elif type == "float32":
            df[key] = df[key].astype(np.float32)
    return df

df = astype(df)

print(df['Label'].value_counts())

df= df[~df['Label'].isin([4, 2, 5, 6])]

df['Label'] = df['Label'].replace(3, 2)
df['Label'] = df['Label'].replace(7, 3)
df['Label'] = df['Label'].replace(8, 4)

# df['Binary Label'] = df['Label'].apply(lambda x: 1 if x != 0 else 0)

print(df['Label'].value_counts())
df.to_parquet(dir_out)

{'Src IP': <class 'numpy.float32'>, 'Src Port': <class 'numpy.int8'>, 'Dst IP': <class 'numpy.float32'>, 'Dst Port': <class 'numpy.int8'>, 'Protocol': <class 'numpy.int8'>, 'Flow Duration': <class 'numpy.float32'>, 'Total Fwd Packet': <class 'numpy.float32'>, 'Total Bwd packets': <class 'numpy.float32'>, 'Total Length of Fwd Packet': <class 'numpy.float32'>, 'Total Length of Bwd Packet': <class 'numpy.float32'>, 'Fwd Packet Length Max': <class 'numpy.float32'>, 'Fwd Packet Length Min': <class 'numpy.float32'>, 'Fwd Packet Length Mean': <class 'numpy.float32'>, 'Fwd Packet Length Std': <class 'numpy.float32'>, 'Bwd Packet Length Max': <class 'numpy.float32'>, 'Bwd Packet Length Min': <class 'numpy.float32'>, 'Bwd Packet Length Mean': <class 'numpy.float32'>, 'Bwd Packet Length Std': <class 'numpy.float32'>, 'Flow Bytes/s': <class 'numpy.float32'>, 'Flow Packets/s': <class 'numpy.float32'>, 'Flow IAT Mean': <class 'numpy.float32'>, 'Flow IAT Std': <class 'numpy.float32'>, 'Flow IAT Max':

In [1]:
# ===== CHIA BATCH INCREMENTAL LEARNING ===== #

import os, sys
from pathlib import Path

import pandas as pd 
import dask.dataframe as dd

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

import hashlib  
import ipaddress
import json
import glob
import gc

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

dtypes = {}    
with open('features.json') as json_file:
    data = json.load(json_file)
    for key, type in data.items():
        if type == "int8":
            dtypes[key]= np.int8
        elif type == "float32":
            dtypes[key] = np.float32
    
    json_file.close()

print(dtypes)

def astype(df):
    for key, type in data.items():
        if type == "int8":
            df[key] = df[key].astype(np.int8)
        elif type == "float32":
            df[key] = df[key].astype(np.float32)
    return df

def splitIL(input, output):
    
    # --  Ép kiểu - Giảm RAM -- 
    df = pd.read_parquet(input)
    df = astype(df)
    
    classes = sorted(df['Label'].unique())
    print(f"Số nhãn có trong dữ liệu gốc: {classes}")
    
    # config
    B_total = 400000
    N_new = 50000

    replay_per_class = lambda c: int(B_total / c)
    
    base_classes = [0, 1, 2]
    sample0 = (
        df[df["Label"].isin(base_classes)]
        .groupby("Label")
        .apply(lambda x: x.sample(B_total))
        .reset_index(drop=True)
    )
    
    sample0 = shuffle(sample0)
    print(sample0["Label"].value_counts())
    sample0.to_parquet(output[0], index=False)
    print(f"✅ Base phase: {base_classes} → {output[0]}")
    
    del(sample0)
    gc.collect()
    
    outfileIndex = 1 
    old_cls = base_classes
    print(old_cls)
    for new_cls in range(3,5):
        E = replay_per_class(len(old_cls))

        print(f"\n--- Increment {new_cls} ---")
        print(f"Old classes: {old_cls}")
        print(f"New class: {new_cls}")
        print(f"Replay mỗi lớp: {E} | Lớp mới: {N_new}")

        # Lấy lại mẫu từ tất cả lớp đã học
        old_samples = (
            df[df["Label"].isin(old_cls)]
            .groupby("Label")
            .apply(lambda x: x.sample(E))
            .reset_index(drop=True)
        )

        # Mẫu mới
        new_samples = df[df["Label"] == new_cls].sample(N_new)
        
        old_samples = shuffle(pd.concat([old_samples, new_samples], ignore_index=True))
        print(old_samples["Label"].value_counts())
        old_samples.to_parquet(output[outfileIndex], index=False)
        
        outfileIndex +=1
        
        # Biến lớp vừa lấy thành chung với lớp cũ
        old_cls.append(new_cls)
        print(old_cls)
        
        del old_samples, new_samples
        gc.collect()
    
if __name__ == "__main__":
    dir_in = "C:/Users/hoang/Documents/Dataset_KLTN/ciciot2023_extracted/merge-processed/merge1.3/part.1.parquet"
    dir_out = [f"C:/Users/hoang/Documents/Dataset_KLTN/ciciot2023_extracted/merge-processed/Incremental_1.3/session{i}.parquet" for i in range(0, 3)]

    print(dir_out)
    splitIL(dir_in, dir_out)

    gc.collect()

{'Src IP': <class 'numpy.float32'>, 'Src Port': <class 'numpy.int8'>, 'Dst IP': <class 'numpy.float32'>, 'Dst Port': <class 'numpy.int8'>, 'Protocol': <class 'numpy.int8'>, 'Flow Duration': <class 'numpy.float32'>, 'Total Fwd Packet': <class 'numpy.float32'>, 'Total Bwd packets': <class 'numpy.float32'>, 'Total Length of Fwd Packet': <class 'numpy.float32'>, 'Total Length of Bwd Packet': <class 'numpy.float32'>, 'Fwd Packet Length Max': <class 'numpy.float32'>, 'Fwd Packet Length Min': <class 'numpy.float32'>, 'Fwd Packet Length Mean': <class 'numpy.float32'>, 'Fwd Packet Length Std': <class 'numpy.float32'>, 'Bwd Packet Length Max': <class 'numpy.float32'>, 'Bwd Packet Length Min': <class 'numpy.float32'>, 'Bwd Packet Length Mean': <class 'numpy.float32'>, 'Bwd Packet Length Std': <class 'numpy.float32'>, 'Flow Bytes/s': <class 'numpy.float32'>, 'Flow Packets/s': <class 'numpy.float32'>, 'Flow IAT Mean': <class 'numpy.float32'>, 'Flow IAT Std': <class 'numpy.float32'>, 'Flow IAT Max':

C:\Users\hoang\AppData\Local\Temp\ipykernel_10748\1651460180.py:65: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[df["Label"].isin(base_classes)]


Label
1    400000
2    400000
0    400000
Name: count, dtype: int64
✅ Base phase: [0, 1, 2] → C:/Users/hoang/Documents/Dataset_KLTN/ciciot2023_extracted/merge-processed/Incremental_1.3/session0.parquet
[0, 1, 2]

--- Increment 3 ---
Old classes: [0, 1, 2]
New class: 3
Replay mỗi lớp: 133333 | Lớp mới: 50000


C:\Users\hoang\AppData\Local\Temp\ipykernel_10748\1651460180.py:92: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[df["Label"].isin(old_cls)]


Label
2    133333
1    133333
0    133333
3     50000
Name: count, dtype: int64
[0, 1, 2, 3]

--- Increment 4 ---
Old classes: [0, 1, 2, 3]
New class: 4
Replay mỗi lớp: 100000 | Lớp mới: 50000


C:\Users\hoang\AppData\Local\Temp\ipykernel_10748\1651460180.py:92: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[df["Label"].isin(old_cls)]


Label
1    100000
2    100000
3    100000
0    100000
4     50000
Name: count, dtype: int64
[0, 1, 2, 3, 4]


In [2]:
ss1 = pd.read_parquet(outputIL[0])
print(ss1.head(10))

     Src IP  Src Port    Dst IP  Dst Port  Protocol  Flow Duration  \
0  3.232271         1  3.232271         1        17       0.978682   
1  3.232271         2  3.232271         0         6       0.050990   
2  3.232271         2  3.232270         2        17       0.000563   
3  3.232271         1  3.232271         1         6       0.409698   
4  3.232271         2  3.232271         1         6       0.000169   
5  3.232271         1  3.232271         1        17       0.093456   
6  3.232271         1  3.232271         2         6       0.058508   
7  3.232271         1  0.134743         0        17       0.000023   
8  3.232271         1  3.232271         0         6       0.000368   
9  3.232271         1  3.232271         1         6       0.822809   

   Total Fwd Packet  Total Bwd packets  Total Length of Fwd Packet  \
0          1.381890          -0.642676                    2.746758   
1         -0.414813          -0.642676                   -0.478489   
2         -0.414813

In [4]:
# ------- CHIA TRAIN TEST IL ------ #

import os, sys
from pathlib import Path

import pandas as pd 
import dask.dataframe as dd

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

import hashlib  
import ipaddress
import json
import glob
import gc

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

dtypes = {}    
with open('features.json') as json_file:
    data = json.load(json_file)
    for key, type in data.items():
        if type == "int8":
            dtypes[key]= np.int8
        elif type == "float32":
            dtypes[key] = np.float32
    
    json_file.close()

print(dtypes)

def astype(df):
    for key, type in df.dtypes.items():
        # print(f"Key: {key} \t {type}")
        if type == "int8":
            df[key] = df[key].astype(np.int8)
        elif type == "float32":
            df[key] = df[key].astype(np.float32)
            
    return df


if __name__ == '__main__':
    dir_in = [f"C:/Users/hoang/Documents/Dataset_KLTN/ciciot2023_extracted/merge-processed/Incremental_1.3/session{i}.parquet" for i in range(0, 3)]
    dir_out_train = [f"C:/Users/hoang/Documents/Dataset_KLTN/ciciot2023_extracted/merge-processed/Incremental_1.3/session{i}_train.parquet" for i in range(0, 3)]
    dir_out_test = [f"C:/Users/hoang/Documents/Dataset_KLTN/ciciot2023_extracted/merge-processed/Incremental_1.3/session{i}_test.parquet" for i in range(0, 3)]
    # print(dir_out_train)
    
    for index, filePath in enumerate(dir_in):
        df = pd.read_parquet(filePath)
        df = astype(df)
        
        # train 
        trainDF = df.sample(frac=0.85, random_state= 42)
        # test
        df = df.drop(trainDF.index)
        # print(df['Binary Label'].value_counts())
        print(df['Label'].value_counts())
        
        print(f"Curr Index: {index+1}")
        trainDF['Binary Label'] = trainDF['Label'].apply(lambda x : 1 if x > index + 1 else 0)
        df['Binary Label'] = df['Label'].apply(lambda x : 1 if x > index + 1 else 0)
        
        print(df['Binary Label'].value_counts())

        trainDF.to_parquet(dir_out_train[index], index= False)
        df.to_parquet(dir_out_test[index], index=False)
        
        
        del trainDF, df
        gc.collect()

{'Src IP': <class 'numpy.float32'>, 'Src Port': <class 'numpy.int8'>, 'Dst IP': <class 'numpy.float32'>, 'Dst Port': <class 'numpy.int8'>, 'Protocol': <class 'numpy.int8'>, 'Flow Duration': <class 'numpy.float32'>, 'Total Fwd Packet': <class 'numpy.float32'>, 'Total Bwd packets': <class 'numpy.float32'>, 'Total Length of Fwd Packet': <class 'numpy.float32'>, 'Total Length of Bwd Packet': <class 'numpy.float32'>, 'Fwd Packet Length Max': <class 'numpy.float32'>, 'Fwd Packet Length Min': <class 'numpy.float32'>, 'Fwd Packet Length Mean': <class 'numpy.float32'>, 'Fwd Packet Length Std': <class 'numpy.float32'>, 'Bwd Packet Length Max': <class 'numpy.float32'>, 'Bwd Packet Length Min': <class 'numpy.float32'>, 'Bwd Packet Length Mean': <class 'numpy.float32'>, 'Bwd Packet Length Std': <class 'numpy.float32'>, 'Flow Bytes/s': <class 'numpy.float32'>, 'Flow Packets/s': <class 'numpy.float32'>, 'Flow IAT Mean': <class 'numpy.float32'>, 'Flow IAT Std': <class 'numpy.float32'>, 'Flow IAT Max':